In [39]:
import tkinter as tk
import numpy as np
import cv2
from PIL import ImageTk,Image,ImageDraw

#----------------------------------
#get same neuarl network architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten #convert image to flattern layer
model=Sequential()
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(10,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#we are not training it here, use
model.load_weights('FFNN-MNIST.h5')
#-----------------------------------

def event_function(event):
    x=event.x
    y=event.y
    
    x1=x-30
    y1=y-30
    
    x2=x+30
    y2=y+30
    
    canvas.create_oval((x1,y1,x2,y2),fill='black')
    #similar the canvas is written, in the image draw it will be drawn-what we going to apply to neural network
    img_draw.ellipse((x1,y1,x2,y2),fill='white')
    
def save():
    global count
    img_array=np.array(img)
    img_array=cv2.resize(img_array,(28,28))
    
    cv2.imwrite(str(count)+'.jpg',img_array)
    count=count+1

def clear():
    global img,img_draw
    canvas.delete('all')
    img=Image.new('RGB',(500,500),(0,0,0))
    img_draw=ImageDraw.Draw(img)
    
    label_status.config(text='PREDICTED DIGIT:NONE')
    
def predict():   
    #take into numpy array
    img_array=np.array(img)
    #convert into gray
    img_array=cv2.cvtColor(img_array,cv2.COLOR_BGR2GRAY)
    img_array=cv2.resize(img_array,(28,28))
    
    img_array=img_array/255.0
    #need to apply reshape before nn
    img_array=img_array.reshape(1,28,28)
    result=model.predict(img_array)
    label=np.argmax(result,axis=1)
    label_status.config(text='PREDICTED DIGIT:'+str(label))
    
    
count=0
#create main window
win=tk.Tk()

#add widget canvas
canvas=tk.Canvas(win,width=500,height=500,bg='white')
#place canvas in the window
canvas.grid(row=0,column=0,columnspan=4)

button_save=tk.Button(win,text='SAVE',bg='green',fg='white',font='Helvetica 20 bold',command=save)
button_save.grid(row=1,column=0)

button_predict=tk.Button(win,text='PREDICT',bg='blue',fg='white',font='Helvetica 20 bold',command=predict)
button_predict.grid(row=1,column=1)

button_clear=tk.Button(win,text='CLEAR',bg='yellow',fg='white',font='Helvetica 20 bold',command=clear)
button_clear.grid(row=1,column=2)

button_exit=tk.Button(win,text='EXIT',bg='red',fg='white',font='Helvetica 20 bold',command=win.destroy)
button_exit.grid(row=1,column=3)

label_status=tk.Label(win,text='PREDICTED DIGIT: NONE',bg='white',font='Helvetica 24 bold')
label_status.grid(row=2,column=0,columnspan=4)

canvas.bind('<B1-Motion>',event_function)

#in our training dataset bg is black and caharactersis white
img=Image.new('RGB',(500,500),(0,0,0))
img_draw=ImageDraw.Draw(img)

#run the main window
win.mainloop() 